In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import os
os.chdir('D:/Capstone/')

In [3]:
## Import Libraries here
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import array
import pandas as pd
import string
import os
import glob
from time import time
from keras import models
from keras.applications import densenet
import shutil
import re
import tensorflow as tf
import tensorflow_hub as hub
import pickle



Using TensorFlow backend.


In [4]:
# This function is used to open a file
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [6]:
## Variables
# images_zip_path = '/content/drive/My Drive/AIML/Capstone/NLMCXR_png.tgz'
images_unzip_path = 'D:/Capstone/NLMCXR_png/'
# Run the following two steps if you wish to unzip the files only
# ##extract_all(images_zip_path,images_unzip_path)
# shutil.unpack_archive(images_zip_path, images_unzip_path)

# reports_zip_path = '/content/drive/My Drive/AIML/Capstone/NLMCXR_reports.tgz'
reports_unzip_path = 'D:/Capstone/NLMCXR_reports/'
# Run the following two steps if you wish to unzip the files only
# ##extract_all(images_zip_path,images_unzip_path)
# shutil.unpack_archive(reports_zip_path, reports_unzip_path)
directory_in_str=reports_unzip_path+'/ecgen-radiology'
pathlist = Path(directory_in_str).glob('**/*.xml')
glove_dir = '/content/drive/My Drive/AIML/Capstone/Glove/'
# Below path contains all the images
images = images_unzip_path
train_images = images_unzip_path
proj_path = 'D:/Capstone/'
chexnet_weights = '/content/drive/My Drive/AIML/Capstone/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

## Pre-Processing

In [7]:
## Extract required XML attributes into a Dataframe here

result=[]
for path in pathlist:
     
     path_in_str = str(path)
     with open(path_in_str, "r") as f:
    
        contents = f.read()

        soup = BeautifulSoup(contents, 'lxml')
        tags = soup.findAll(['abstracttext','parentimage',{'id':True}])
        for t in tags:
          label=t.get("label")
          if label=='FINDINGS':
            f_text=t.text
            
          if label=='IMPRESSION':
            i_text=t.text
            
          if label=='INDICATION':
            ind_text=t.text
           
          for j in t.find_all('figureid'):
            if j.text=='F1':
              id=t["id"]

          
        result.append([path_in_str[43:100],id,f_text,i_text,ind_text])

                


In [8]:
df=pd.DataFrame(result,columns=["XML_File_Name","Image_File_name","Findings","Impressions","Indication"])

In [9]:
df.head()

,XML_File_Name,Image_File_name,Findings,Impressions,Indication
0,1.xml,CXR1_1_IM-0001-3001,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,Positive TB test
1,10.xml,CXR10_IM-0002-1001,The cardiomediastinal silhouette is within nor...,No acute cardiopulmonary process.,"XXXX-year-old male, chest pain."
2,100.xml,CXR100_IM-0002-1001,Both lungs are clear and expanded. Heart and m...,No active disease.,
3,1000.xml,CXR1000_IM-0003-1001,There is XXXX increased opacity within the rig...,1. Increased opacity in the right upper lobe w...,"XXXX-year-old male, XXXX."
4,1001.xml,CXR1001_IM-0004-1001,Interstitial markings are diffusely prominent ...,Diffuse fibrosis. No visible focal acute disease.,"dyspnea, subjective fevers, arthritis, immigra..."


In [11]:
# Indication have blank values (530 rows)
df[df["Indication"]==''].count()

XML_File_Name      90
Image_File_name    90
Findings           90
Impressions        90
Indication         90
dtype: int64

In [12]:
# Findings have blank values (530 rows)
df[df["Findings"]==''].count()

XML_File_Name      530
Image_File_name    530
Findings           530
Impressions        530
Indication         530
dtype: int64

In [13]:
# Update Findings --> Fill blank values with Impressions
df["Findings"]=np.where(df["Findings"]=='',df["Impressions"],df["Findings"])

In [14]:
df[df["Findings"]==''].count()

XML_File_Name      28
Image_File_name    28
Findings           28
Impressions        28
Indication         28
dtype: int64

In [15]:
## Remove Blank reports -- 28 Reports
df_final=df[df["Findings"]!='']

In [16]:
df_final.head()

,XML_File_Name,Image_File_name,Findings,Impressions,Indication
0,1.xml,CXR1_1_IM-0001-3001,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,Positive TB test
1,10.xml,CXR10_IM-0002-1001,The cardiomediastinal silhouette is within nor...,No acute cardiopulmonary process.,"XXXX-year-old male, chest pain."
2,100.xml,CXR100_IM-0002-1001,Both lungs are clear and expanded. Heart and m...,No active disease.,
3,1000.xml,CXR1000_IM-0003-1001,There is XXXX increased opacity within the rig...,1. Increased opacity in the right upper lobe w...,"XXXX-year-old male, XXXX."
4,1001.xml,CXR1001_IM-0004-1001,Interstitial markings are diffusely prominent ...,Diffuse fibrosis. No visible focal acute disease.,"dyspnea, subjective fevers, arthritis, immigra..."


In [1]:
# This step is just to check the data. This need not be executed
# df_final[df_final['Findings'].str.contains('5XXXX', regex=True)]

In [58]:
# Create dictionary of Indication sentences
s_indication=dict(zip(df_final.Image_File_name,df_final.Indication))
# Create dictionary of Impression sentences
s_impression=dict(zip(df_final.Image_File_name,df_final.Impressions))
# Create dictionary of Findings sentences
s_findings=dict(zip(df_final.Image_File_name,df_final.Findings))
# Create dictionary of Findings Words
w_findings=dict(zip(df_final.Image_File_name,df_final.Findings))
# Create dictionary of Indication Words
w_indication=dict(zip(df_final.Image_File_name,df_final.Indication))
# Create dictionary of Impression Words
w_impression=dict(zip(df_final.Image_File_name,df_final.Impressions))

In [21]:
list(s_findings.keys())[:5]

['CXR1_1_IM-0001-3001',
 'CXR10_IM-0002-1001',
 'CXR100_IM-0002-1001',
 'CXR1000_IM-0003-1001',
 'CXR1001_IM-0004-1001']

In [23]:
## Function to Clean up sentences or words
remove = string.punctuation
remove = remove.replace(".", "")

def s_clean_up(anything):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', remove)
	for key, desc_list in anything.items():

		desc = desc_list
		
		# convert to lower case
		desc = desc.lower()
		# remove punctuation from each token
		desc = desc.replace('/', ' or ')
		desc = desc.replace('-', ' ')
#		desc = desc.replace('<BR>', ' ')
		desc = desc.replace('<br>', ' ')
		desc = desc.replace('&gt;]', '')
		desc = desc.replace('[&lt;', '')
		desc = re.sub(r'\(.*?\)', '', desc)
# Add space after period where statements don't start after period        
		desc = re.sub(r'(?<=[.])(?=[^\d\s])', r' ', desc)
		desc = desc.translate(table)
# 		desc = desc.replace(' 0', 'zero ')
# 		desc = desc.replace(' 1', 'one ')
# 		desc = desc.replace(' 2', 'two ')
# 		desc = desc.replace(' 3', 'three ')
# 		desc = desc.replace(' 4', 'four ')
# 		desc = desc.replace(' 5', 'five ')
# 		desc = desc.replace(' 6', 'six ')
# 		desc = desc.replace(' 7', 'seven ')
# 		desc = desc.replace(' 8', 'eight ')
# 		desc = desc.replace(' 9', 'nine ')
# 		desc = desc.replace('.', '.')        
##		desc = re.sub('\d', 'XX', desc)
		# store as string
		anything[key] =  desc


In [ ]:
# Clean up Sentences and Words of Findings
s_clean_up(s_findings)
s_clean_up(w_findings)

# Clean up Sentences and Words of Indication
s_clean_up(s_indication)
s_clean_up(w_indication)

# Clean up Sentences and Words of Impression
s_clean_up(s_impression)
s_clean_up(w_impression)

In [25]:
s_findings['CXR2192_IM-0802-2002']

'no focal lung consolidation. no pneumothorax or pleural effusion. heart size and pulmonary vascularity are within normal limits. minimal degenerative changes of the thoracic spine. the previously described xxxx deformity in the midthoracic spine is again seen. there is subcutaneous shunt catheter tubing along the anterior chest wall'

In [62]:
w_findings['CXR1040_IM-0032-2001']

'Heart size is normal. Mediastinal width is within normal limits for supine AP technique. No edema. No focal infiltrate. No pleural effusion or pneumothorax. Right hilar and right lung base calcifications are compatible with old granulomatous disease. There is a very mild anterior wedge deformity of a midthoracic vertebrae, possibly T7, age-indeterminate. Correlate for midthoracic tenderness. No displaced, acute rib fractures are identified.'

In [63]:
w_indication['CXR1_1_IM-0001-3001']

'Positive TB test'

In [64]:
w_impression['CXR1_1_IM-0001-3001']

'normal chest x xxxx.'

In [65]:
# convert the loaded descriptions into a vocabulary of sentences
def s_to_vocabulary(anything):
	# build a list of all description strings
	all_desc = set()
	for key in anything.keys():
		[all_desc.update(anything[key].split('. ')) ]
	return all_desc


# summarize vocabulary
#impressions_vocabulary = to_vocabulary(impressions)
#print('Original Impressions Vocabulary Size: %d' % len(impressions_vocabulary))

Original Findings Sentences Vocabulary Size: 6736


In [ ]:
# summarize findings vocabulary
s_findings_vocabulary = s_to_vocabulary(s_findings)
print('Original Findings Sentences Vocabulary Size: %d' % len(s_findings_vocabulary))

In [66]:
sv=list(s_findings_vocabulary)

In [70]:
# convert the loaded descriptions into a vocabulary of words
def w_to_vocabulary(anything):
	# build a list of all description strings
	all_desc = set()
	for key in anything.keys():
		[all_desc.update(anything[key].split(' ')) ]
	return all_desc


# summarize vocabulary
#impressions_vocabulary = to_vocabulary(impressions)
#print('Original Impressions Vocabulary Size: %d' % len(impressions_vocabulary))

Original Findings Word Vocabulary Size: 3192


In [ ]:
# summarize findings vocabulary
w_findings_vocabulary = w_to_vocabulary(w_findings)
print('Original Findings Word Vocabulary Size: %d' % len(w_findings_vocabulary))
wl=list(w_findings_vocabulary)

In [71]:
# summarize indication vocabulary
w_indication_vocabulary = w_to_vocabulary(w_indication)
print('Original Indication Word Vocabulary Size: %d' % len(w_indication_vocabulary))
ind_wl=list(w_indication_vocabulary)

In [72]:
# summarize impression vocabulary
w_impression_vocabulary = w_to_vocabulary(w_impression)
print('Original Impression Word Vocabulary Size: %d' % len(w_impression_vocabulary))
imp_wl=list(w_impression_vocabulary)

In [76]:
## Remove periods attached to words from the findings vocabulary
for i,r in enumerate(wl):
    if len(r)>1:
        if str(r)[len(r)-1:len(r)] == '.':
            wl[i]=wl[i].replace('.', '')

In [78]:
## Remove periods attached to words from the indication vocabulary
for i,r in enumerate(ind_wl):
    if len(r)>1:
        if str(r)[len(r)-1:len(r)] == '.':
            ind_wl[i]=ind_wl[i].replace('.', '')

In [79]:
## Remove periods attached to words from the impression vocabulary
for i,r in enumerate(imp_wl):
    if len(r)>1:
        if str(r)[len(r)-1:len(r)] == '.':
            imp_wl[i]=imp_wl[i].replace('.', '')

In [81]:
# save descriptions to file, one per line
def save_anything(anything, filename):
	lines = list()
	for key, desc_list in anything.items():
		lines.append(key + ' ' + desc_list)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_anything(s_findings, 's_findings.txt')
save_anything(w_findings, 'w_findings.txt')
save_anything(w_indication, 'w_indication.txt')
save_anything(w_impression, 'w_impression.txt')

In [84]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text



In [85]:
# load clean descriptions into memory and add the required startseq and endseq tokens here. 
# This is required to get the sentence start and end points
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if train_images+image_id+'.png' in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_s_findings = load_clean_descriptions('s_findings.txt', train)
print('Descriptions: train=%d' % len(train_s_findings))

Descriptions: train=3668


In [88]:
 train_s_findings['CXR1220_IM-0148-1001']

['startseq There is hyperexpansion, hyperlucency of both lungs, as well as flattening of the diaphragm consistent with chronic emphysematous lung disease. No focal consolidation, pneumothorax, or large pleural effusion identified (blunting of costophrenic recesses bilaterally may represent small effusions or pleural thickening/scar. Stable calcified mediastinal and hilar lymph XXXX and a left basilar calcified granuloma. Acute osseous abnormality. The mediastinal silhouette stable. endseq']

In [57]:
# Pickle the output files
def pickle_op(typ,anything):
    path = proj_path#'./processed_data'
    pickle_file = path+typ+".pickle"
    pickle.dump(anything, open(pickle_file, 'wb'))

    


In [89]:
# Findings Word Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('word_vocab',wl)


# Findings Sentence Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('sent_vocab',sv)


# Indication Word Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('indication_vocab',ind_wl)


# Indication Sentence Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('indication_keys',s_indication)


# Impression Sentence Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('impression_keys',s_impression)


# Impression Word Vocabulary. This is required first time only. Uncomment and execute if necessary.
# pickle_op('impression_vocab',imp_wl)